<a href="https://colab.research.google.com/github/Odima-dev/Data-Science-and-Machine-Learning/blob/main/TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**# Problem 1: Looking back on scratch**

Looking back at the scratches so far, when using scratch implementation of neural networks had to manually implement:



1. Weight initialization
2. I had to define forward propagation
3. I needed to calculate loss, including mean squared error and cross-entropy.
4. I implemeted Backpropagation
5. Implemted gradient parameter updates
6. I needed an epoch loop
7. I implemeted Mini-batch processing
8. I also did accuracy evaluation
9. I also performed preprocessing of data data load, split, and normalize.

**#Problem 2: Consider the correspondence between scratch and TensorFlow**

This is how TensorFlow implements the following:

1. Weight initialization - uses `tf.Variable(tf.random_normal(...))`
2. Forward propagation - done via `tf.matmul`, `tf.add`, `tf.nn.relu`
3. Loss calculation - achieved by `tf.nn.sigmoid_cross_entropy_with_logits`
4. Backpropagation - one by `optimizer.minimize(loss_op)`
5. Parameter update(gradient) - achieved by `train_op = optimizer.minimize(...)`
6. Epoch loop - done via `for epoch in range(num_epochs)`
7. Mini-batch handling - is `GetMiniBatch` class with `for x, y in ...`
8. Accuracy calculation - implemeted via `tf.equal(...)`, `tf.reduce_mean(...)`
9. Data preprocessing  - achieved by `pandas`, `numpy`, and `sklearn`

In [2]:
# Problem 3: Create a model of Iris using all three types of objective variables
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Loading Iris dataset
from google.colab import files
uploaded = files.upload()

df = pd.read_csv("Iris.csv")
X = df[["SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"]].values
y = df["Species"].values

# One-hot encode labels
encoder = LabelBinarizer()
y = encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# Hyperparameters
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_input = X_train.shape[1]
n_hidden1, n_hidden2 = 50, 100
n_classes = 3
n_samples = X_train.shape[0]

# Placeholder
X_ph = tf.placeholder(tf.float32, [None, n_input])
Y_ph = tf.placeholder(tf.float32, [None, n_classes])

# Weights and biases
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
    'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
    'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden1])),
    'b2': tf.Variable(tf.random_normal([n_hidden2])),
    'b3': tf.Variable(tf.random_normal([n_classes]))
}

# Model
def model(x):
    l1 = tf.nn.relu(tf.add(tf.matmul(x, weights['w1']), biases['b1']))
    l2 = tf.nn.relu(tf.add(tf.matmul(l1, weights['w2']), biases['b2']))
    return tf.matmul(l2, weights['w3']) + biases['b3']

logits = model(X_ph)

# Loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_ph, logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y_ph, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

# Training
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        for i in range(0, n_samples, batch_size):
            x_batch = X_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]
            sess.run(optimizer, feed_dict={X_ph: x_batch, Y_ph: y_batch})

        if epoch % 10 == 0:
            val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X_ph: X_val, Y_ph: y_val})
            print(f"Epoch {epoch}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.3f}")

    test_acc = sess.run(accuracy, feed_dict={X_ph: X_test, Y_ph: y_test})
    print("Test Accuracy:", test_acc)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Saving Iris.csv to Iris.csv
Saving train.csv to train.csv
Epoch 0, val_loss: 266.0496, val_acc: 0.333
Epoch 10, val_loss: 2.2288, val_acc: 0.792
Epoch 20, val_loss: 1.2974, val_acc: 0.917
Epoch 30, val_loss: 1.0585, val_acc: 0.917
Epoch 40, val_loss: 0.8790, val_acc: 0.917
Epoch 50, val_loss: 0.9280, val_acc: 0.958
Epoch 60, val_loss: 1.1742, val_acc: 0.917
Epoch 70, val_loss: 1.0960, val_acc: 0.917
Epoch 80, val_loss: 1.0950, val_acc: 0.917
Epoch 90, val_loss: 1.0946, val_acc: 0.917
Test Accuracy: 1.0


In [3]:
# Problem 4: Create a model of House Prices
# Loading dataset
df = pd.read_csv("train.csv")
df = df[["GrLivArea", "YearBuilt", "SalePrice"]].dropna()
X = df[["GrLivArea", "YearBuilt"]].values
y = df["SalePrice"].values.reshape(-1, 1)

# Normalizing
X = (X - X.mean(axis=0)) / X.std(axis=0)
y = (y - y.mean()) / y.std()

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

# Placeholder
X_ph = tf.placeholder(tf.float32, [None, 2])
Y_ph = tf.placeholder(tf.float32, [None, 1])

# Weights and biases
W1 = tf.Variable(tf.random_normal([2, 64]))
b1 = tf.Variable(tf.zeros([64]))
W2 = tf.Variable(tf.random_normal([64, 1]))
b2 = tf.Variable(tf.zeros([1]))

# Model
l1 = tf.nn.relu(tf.matmul(X_ph, W1) + b1)
output = tf.matmul(l1, W2) + b2

# Loss and optimizer
loss = tf.reduce_mean(tf.square(output - Y_ph))
optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)

# Training
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(100):
        for i in range(0, len(X_train), batch_size):
            x_batch = X_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]
            sess.run(optimizer, feed_dict={X_ph: x_batch, Y_ph: y_batch})

        if epoch % 10 == 0:
            val_loss = sess.run(loss, feed_dict={X_ph: X_val, Y_ph: y_val})
            print(f"Epoch {epoch}, val_loss: {val_loss:.4f}")

    test_loss = sess.run(loss, feed_dict={X_ph: X_test, Y_ph: y_test})
    print("Test Loss:", test_loss)

Epoch 0, val_loss: 2.2282
Epoch 10, val_loss: 0.3786
Epoch 20, val_loss: 0.3799
Epoch 30, val_loss: 0.4176
Epoch 40, val_loss: 0.4191
Epoch 50, val_loss: 0.4124
Epoch 60, val_loss: 0.3923
Epoch 70, val_loss: 0.3768
Epoch 80, val_loss: 0.3774
Epoch 90, val_loss: 0.3783
Test Loss: 0.350715


In [7]:
# Problem 5: Create a model of MNIST
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Loading MNIST from tf.keras
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalizing and reshaping
x_train = x_train.reshape(-1, 784) / 255.0
x_test  = x_test.reshape(-1, 784) / 255.0

# One-hot encode labels
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse_output=False)
y_train = enc.fit_transform(y_train.reshape(-1, 1))
y_test  = enc.transform(y_test.reshape(-1, 1))

# Splitting validation set
x_val = x_train[-5000:]
y_val = y_train[-5000:]
x_train = x_train[:-5000]
y_train = y_train[:-5000]

# Placeholders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

# Model parameters
W1 = tf.Variable(tf.random_normal([784, 128]))
b1 = tf.Variable(tf.zeros([128]))
W2 = tf.Variable(tf.random_normal([128, 10]))
b2 = tf.Variable(tf.zeros([10]))

# Building model
l1 = tf.nn.relu(tf.matmul(X, W1) + b1)
logits = tf.matmul(l1, W2) + b2

# Loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss_op)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Training
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(10):
        for i in range(0, len(x_train), 100):
            x_batch = x_train[i:i+100]
            y_batch = y_train[i:i+100]
            sess.run(optimizer, feed_dict={X: x_batch, Y: y_batch})

        val_acc = sess.run(accuracy, feed_dict={X: x_val, Y: y_val})
        print(f"Epoch {epoch+1}, Validation Accuracy: {val_acc:.4f}")

    test_acc = sess.run(accuracy, feed_dict={X: x_test, Y: y_test})
    print(f"Test Accuracy: {test_acc:.4f}")


Epoch 1, Validation Accuracy: 0.8188
Epoch 2, Validation Accuracy: 0.8710
Epoch 3, Validation Accuracy: 0.8918
Epoch 4, Validation Accuracy: 0.9076
Epoch 5, Validation Accuracy: 0.9154
Epoch 6, Validation Accuracy: 0.9220
Epoch 7, Validation Accuracy: 0.9270
Epoch 8, Validation Accuracy: 0.9302
Epoch 9, Validation Accuracy: 0.9332
Epoch 10, Validation Accuracy: 0.9356
Test Accuracy: 0.9212
